### 1. Instalamos las librerias necesarias

In [ ]:
#pip install --upgrade pandas openpyxl xlrd

In [ ]:
#Instalamos de ser necesario, caso contrario omitir este paso
#pip install requests
#pip install pandas, openpyxl

### 2. Importamos las librerias necesarias

In [ ]:
import pandas as pd
import numpy as np
import requests
import os
import win32com.client as win32
import re

### 3. Eligimos el tipo de empresa del Sistema Financiero a revisar

In [ ]:
t_empr = input(
    '''¿Que tipo de empresa desea analizar: 
    0: Banca Múltiple 
    1: Empresas Financieras
    2: Caja Municipal 
    3: Caja Rural
    4: Empresas de Créditos

    
    '''
)

### 4. Eligimos el periodo a descargar

#### 4.1 Eligimos los años

In [ ]:
year_start = int(input(
    ''' ¿Desde qué año se va a consultar:
 (Ejem: 2023)

'''))

year_end = int(input(
    ''' ¿Hasta qué año se va a consultar:
 (Ejem: 2024)


'''))

#### 4.2 Eligimos los meses

In [ ]:
month1 = int(input(
    ''' ¿Desde qué mes se va a consultar:
 (Ejem: 1, para establecer hasta el mes de enero)


'''))

month2 = int(input(
    ''' ¿Hasta qué mes se va a consultar:
 (Ejem: 11, para establecer hasta el mes de noviembre)


'''))

### 5. Seleccionamos el archivo a descargar

In [ ]:
selec_archivo = input(
    ''' Seleccione el archivo a consultar:
    0: Estados Financieros
    1: Req de Patrimonio Efec y Ratio Capital global
    2: Cred Directos según situación
    3: Ratios Morosidad dias cumplimiento
    4: Estructura de Créditos Directos y Contingentes según Categoría de Riesgo del Deudor
    5: Número de Deudores según Tipo de Crédito
    6: Morosidad por tipo de crédito y modalidad
    7: Estructura de Créditos Directos, Indirectos y Contingentes por Tipo de Crédito y Categoría de Riesgo del Deudor
    8: Créditos Directos según Tipo de Crédito y Situación

    
    ''')
tipo_empresa_descargar = {'0': {'0':'B-2201','1': 'B-3101', '2':'C-1101', '3':'C-2101', '4':'C-4103'},
                          '1': {'0':'B-2402','1': 'B-3302', '2':'C-1252', '3':'C-2257', '4':'C-4241'}, 
                          '2': {'0':'B-2315','1': 'B-3219', '2':'C-1207', '3':'C-2207', '4':'C-4208'},
                          '3': {'0':'B-220512','1': 'B-3230', '2':'C-1230', '3':'C-2230', '4':'C-4225'},
                          '4': {'0':'B-230802','1': 'B-3205', '2':'C-120201', '3':'C-220201', '4':'C-4201'},
                          '5': {'0':'B-230803','1': 'B-3218', '2':'C-1231', '3':'C-2231', '4':'C-4226'},
                          '6': {'0':'B-2362','1': 'B-3257', '2':'C-1242', '3':'C-2247', '4':'C-4234'},
                          '7': {'0':'B-230802','1': 'B-3271', '2':'C-1233', '3':'C-2233', '4':'C-4227'},
                          '8': {'0':'B-2334','1': 'B-3220', '2':'C-1228', '3':'C-2228', '4':'C-4223'}
                         }

archivo_descargado = {'0': 'EEFF', '1': 'RPF_RCG', '2': 'CD_SITUACION','3':'RM_dias','4':'ECD_CRD','5':'ND',
                      '6':'MORA_TC_MOD','7':'ECD_TC_CRD','8':'CD_TC_SIT'}

In [ ]:
# Función para guardar un archivo como .XLS usando Excel
def guardar_como_xls(ruta_archivo):
    try:
        # Iniciar Excel
        excel = win32.Dispatch('Excel.Application')
  
        excel.Visible = False  # No mostrar la interfaz de Excel
        excel.DisplayAlerts = False  # Deshabilitar alertas (incluyendo la de guardar)

        # Abrir el archivo
        workbook = excel.Workbooks.Open(ruta_archivo)

        # Guardar como .XLS (mismo nombre y ubicación)
        workbook.SaveAs(ruta_archivo, FileFormat=56)  # 56 es el código para .XLS
        workbook.Close(SaveChanges=False)  # Cerrar sin guardar cambios adicionales
        excel.Quit()
        print(f"Archivo guardado como {ruta_archivo}")
        return True
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")
        return False
    finally:
        # Asegurarse de que Excel se cierre correctamente
        if 'excel' in locals():
            excel.Quit()

def limpiar_texto(texto):
    if isinstance(texto, str):
        return ' '.join([''.join(palabra.split()) for palabra in texto.split()])
    return texto  # si no es texto, no hacer nada

def limpiar_nombre(texto):
    # Convertir a string por seguridad
    texto = str(texto)
    # 1. Eliminar todo lo que esté desde el primer asterisco en adelante
    texto = texto.split('*')[0]
    # 2. Eliminar cualquier número y símbolo (excepto letras y espacios)
    texto = re.sub(r'[^A-Za-zÁÉÍÓÚÑáéíóúñ ]', '', texto)
    # 3. Reemplazar múltiples espacios por uno solo
    texto = re.sub(r'\s+', ' ', texto)
    # 4. Eliminar espacios al inicio y al final
    texto = texto.strip()
    return texto


### 6. Descargamos los archivos de la SBS

In [ ]:
carpeta = {'0': 'BM', '1': 'EF', '2': 'CMAC', '3': 'CRAC' , '4': 'EC' }
ruta_base = f"D:\\SBS\\EMP SF\\{carpeta[t_empr]}"
tipo_empresa = {'0': 'Banca Múltiple', '1': 'Empresas Financieras', '2': 'Caja Municipal', '3': 'Caja Rural' , '4': 'Empresas de Créditos' }
mes_corto = {0:'en', 1:'fe', 2:'ma', 3:'ab', 4:'my', 5:'jn', 6:'jl', 7:'ag', 8:'se', 9:'oc', 10:'no', 11:'di'}

ruta_inicial = 'https://intranet2.sbs.gob.pe/estadistica/financiera/'
meses = {0:'Enero',1:'Febrero',2:'Marzo',3:'Abril',4:'Mayo',5:'Junio',6:'Julio',7:'Agosto',8:'Setiembre',9:'Octubre',10:'Noviembre',11:'Diciembre'}

#RUTA DONDE SE GUARDARAN LOS ARCHIVOS

# CREAMOS LA CARPETA DE GUARDADO CON EL NOMBRE DEL AÑO
for year in range(int(year_start),int(year_end) +1):
    year1 = str(year)
    nombre_carpeta = tipo_empresa[t_empr]+'-' + archivo_descargado[selec_archivo] +'_'+year1
    ruta_guardado_anio = os.path.join(ruta_base, nombre_carpeta )
    os.makedirs(ruta_guardado_anio, exist_ok=True)
    if year < int(year_end):
        for m in range(12):
            nombre_archivo = f'{tipo_empresa_descargar[selec_archivo][t_empr]}-{mes_corto[m]}{year1}.XLS'
            url_descarga = f'{ruta_inicial}{year1}/{meses[m]}/{nombre_archivo}'
            # Descargar el archivo
            response = requests.get(url_descarga, stream=True)
            # Verificar si la descarga fue exitosa
            if response.status_code == 200:
                ruta_guardado = os.path.join(ruta_guardado_anio, nombre_archivo)  # Guardar en la carpeta creada
                with open(ruta_guardado, "wb") as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                print(f"Archivo descargado exitosamente: {ruta_guardado}")
            else:
                print(f"No se pudo descargar el archivo. Código de estado: {response.status_code}")
    else:
        for m in range(int(month)):
            nombre_archivo = f'{tipo_empresa_descargar[selec_archivo][t_empr]}-{mes_corto[m]}{year1}.XLS'
            url_descarga = f'{ruta_inicial}{year1}/{meses[m]}/{nombre_archivo}'
            # Descargar el archivo
            response = requests.get(url_descarga, stream=True)
            # Verificar si la descarga fue exitosa
            if response.status_code == 200:
                ruta_guardado = os.path.join(ruta_guardado_anio, nombre_archivo)  # Guardar en la carpeta creada
                with open(ruta_guardado, "wb") as file:
                    for chunk in response.iter_content(1024):
                        file.write(chunk)
                print(f"Archivo descargado exitosamente: {ruta_guardado}")
            else:
                print(f"No se pudo descargar el archivo. Código de estado: {response.status_code}")

### 7. EXTRACCION CUENTAS DE ESTADOS FINANCIEROS

#### 7.1 ESTADO DE GANANCIAS Y PERDIDAS

In [ ]:
#SOLO PARA ESTADOS FINANCIEROS - GYP
tipo_empresa = {'0': 'Banca Múltiple', '1': 'Empresas Financieras', '2': 'Caja Municipal', '3': 'Caja Rural' , 
                '4': 'Empresas de Créditos' }
carpeta = {'0': 'BM', '1': 'EF', '2': 'CMAC', '3': 'CRAC' , '4': 'EC' }
mes_corto = {0:'en', 1:'fe', 2:'ma', 3:'ab', 4:'my', 5:'jn', 6:'jl', 7:'ag', 8:'se', 9:'oc', 10:'no', 11:'di'}
pd.set_option('display.float_format', '{:.2f}'.format)
ctas_extraer = ['INGRESOS FINANCIEROS','GASTOS FINANCIEROS','GASTOS ADMINISTRATIVOS',
                'RESULTADO NETO DEL EJERCICIO', 
                'UTILIDAD (PÉRDIDA) ANTES DE PARTICIPACIONES E IMPUESTO A LA RENTA',
                'UTILIDAD (PÉRDIDA) NETA',
                'PROVISIONES INCOBRABILIDAD DE CRÉDITOS',
                'PROVISIONES. DEPRECIACIÓN Y AMORTIZACIÓN',
                 'UTILIDAD ( PÉRDIDA ) NETA']
hoja_extraer = 'gyp'

ruta_base = f"D:\\SBS\\EMP SF\\{carpeta[t_empr]}"
data = {}
for year in range(year_start,year_end +1):
    for m in range(12):
        fecha = f"{year}{str(m+1).zfill(2)}01"
        fecha1 = pd.to_datetime(fecha, format='%Y%m%d')
        if (fecha1 >= pd.to_datetime(f'{year_start}-{month1}-01')) and (fecha1 <= pd.to_datetime(f'{year_end}-{month2}-01')):
            nombre_archivo = f'{tipo_empresa_descargar[selec_archivo][t_empr]}-{mes_corto[m]}{year}.XLS' 
            ruta_archivo = f'{ruta_base}\\{tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{year}\\{nombre_archivo}'
            guardar_como_xls(ruta_archivo)
            archivo = pd.read_excel(ruta_archivo, engine='xlrd', sheet_name = 0)
            try:
                print("Archivo leído correctamente después de guardar como .XLS.")
            except Exception as e:
                print(f'Error al cargar {nombre_archivo}:{e}')

            archivo_base= archivo.copy()
            
            if pd.isna(archivo.iloc[0,0]) == True:
                archivo = archivo.iloc[:,1:]
            
            #Eliminamos todos los espacios innecesarios de la primera columna
            archivo.iloc[:, 0] = archivo.iloc[:, 0].astype(str).str.strip().str.replace(r'\s+', ' ', regex=True)
            #EVALUAMOS SI LA HOJA IMPORTADA TIENE LA CUENTA INGRESOS FINANCIEROS
            if archivo[archivo.iloc[:, 0].str.contains("INGRESOS FINANCIEROS", na=False, case=False)].index.tolist() == []:
                archivo = pd.read_excel(ruta_archivo, engine='xlrd', sheet_name = 1)
                archivo_base = archivo.copy()
                if pd.isna(archivo.iloc[0,0]) == True:
                    archivo = archivo.iloc[:,1:]
            ## ==============================================TRABAJANDO CON EL DF IMPORTADO =========================================================
            #####  PARTE 01 : IDENTIFICAR LAS FILAS QUE VAMOS A TRABAJAR
            
            #Aplicamos la función limpiar texto, para eliminar los espacios innecesarios
            archivo = archivo.applymap(limpiar_texto)
            #Eliminamos todos los valores nan del df archivo
            archivo = archivo.dropna(axis=1, how='all')
            #Determinamos el indice de la palabra INGRESOS FINANCIEROS
            indice1 = archivo[archivo.iloc[:, 0].str.contains("INGRESOS FINANCIEROS", na=False, case=False)].index.tolist()
            archivo_base1= archivo.copy()         
            #Colocamos toda la fila del indice del Activo como nombres de columnas
            archivo.columns =archivo.iloc[indice1[0]-3]
            #Cambiamos los nombres de las columnas a MAYUSCULAS 
            archivo.columns = [str(nombre).upper().strip().replace("  ", " ") 
                                if isinstance(nombre, str) 
                                else nombre for nombre in archivo.columns]
                
            archivo.columns = [np.nan if col == 'CONCEPTO' else col for col in archivo.columns]
                
            #Nos quedamos con las filas debajo del indice Activo
            archivo = archivo.iloc[indice1[0]-1:,:]
            #Establecemos una lista de todos los nombres de las columnas
            col_names = archivo.columns.tolist()
            #Agregamos el nombre CUENTAS y quitamos el nombre Activo
            col_names1 = ['CUENTAS'] + col_names[1:]
            archivo.columns = col_names1
            
            archivodd = archivo.copy()
            #Solo nos quedamos con las filas de las cuentas que estan en la lista "ctas_extraer"
            archivo = archivo[archivo.CUENTAS.isin(ctas_extraer)]
           
            #Reseteamos el indice
            archivo = archivo.reset_index(drop=True)

            #####  PARTE 02 : IDENTIFICAR LAS COLUMNAS QUE VAMOS A TRABAJAR
            archivo3 = archivo.copy()
            #Del archivo incial, nos quedamos que las columnas sean la fila de los MN,ME,MT
            archivo2 = archivo_base1.copy()
            archivo2.columns = archivo2.iloc[indice1[0]-2]
            #Establecemos el criterio de seleccion
            nombre_seleccion = 'TOTAL'
            columnas_seleccion = []
            columnas_seleccion = [i for i, col in enumerate(archivo2.columns) if pd.notna(col) and col == nombre_seleccion]
            columnas_conservar = [0] + columnas_seleccion

            ### CASO ESPECIAL: EN CASO FALTE UN ENTIDAD QUE NO TENGA LOS VALORES: MN, ME Ó TOTAL
            #### Primero, verificamos la cantidad de entidades
            entidades = archivo3.columns.tolist()
            entidades = pd.Series(entidades).dropna(how='all').tolist()
            entidades1 = entidades[1:]
            entidades_index = [i for i, col in enumerate(archivo3.columns) if pd.notna(col)]
            #### Segundo, creamos una lista de todo el orden cada entidad en base a lo reportado anteriormente       
            col_MN = [0]
            col_ME = [0]
            col_TOTAL = [0]

            for m in range(1,len(entidades_index)):
                if (archivo_base1.iloc[indice1[0]:indice1[0]+1,entidades_index[m]].values[0] !=0) or pd.notna(archivo_base1.iloc[indice1[0]:indice1[0]+1,entidades_index[m]].values[0]) == True:
                    valor1 = entidades_index[m]
                    col_MN.append(valor1)
                    valor2 = valor1+1
                    col_ME.append(valor2)
                    valor3 = valor2+1
                    col_TOTAL.append(valor3)
                else:
                    valor1 = entidades_index[m]+1
                    col_MN.append(valor1)
                    valor2 = valor1+1
                    col_ME.append(valor2)
                    valor3 = valor2+1
                    col_TOTAL.append(valor2)
            #### Tercero, evaluamos para 
            #Establecemos las columnas seleccionadas
            moneda = {}
            moneda = {'MN':col_MN, 'ME':col_ME, 'TOTAL':col_TOTAL}
            if len(columnas_conservar) < len(moneda[nombre_seleccion]):
                archivo = archivo.iloc[:,moneda[nombre_seleccion]]
            else:
                archivo = archivo.iloc[:,columnas_conservar]

            #####  PARTE 03 : NOS QUEDAMOS CON LAS FILAS Y COLUMNAS SELECCIONADAS
            #Ahora nos quedamos solo con los nombres de las entidades financieras
            archivo.columns = entidades
            archivo['FECHA'] = fecha1
            archivo = archivo.melt(id_vars=['CUENTAS','FECHA'], var_name=tipo_empresa[t_empr], value_name='Valor')
            archivofinal = archivo.pivot_table(index=[tipo_empresa[t_empr],'FECHA'], columns='CUENTAS', values='Valor', aggfunc='sum')
            archivofinal.reset_index(inplace = True)
            archivofinal.iloc[:, 0] = archivofinal.iloc[:, 0].apply(limpiar_nombre)
            data[fecha] = archivofinal

data_total = pd.concat(data.values(),axis=0)
columnas_orden = [data_total.columns[1]] + [data_total.columns[0]] + list(data_total.columns[2:])
data_total = data_total[columnas_orden]
ruta_descarga = f"C:\\Users\\USER\\Documents\\GITHUB\\"
data_generada = f'{ruta_descarga}{tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{hoja_extraer}_{nombre_seleccion}_{year_start}.{month1:02}-{year_end}.{month2:02}.xlsx'
data_total.to_excel(data_generada, index=False)
try:
    print(f'Archivo {tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{hoja_extraer}_{year_start}-{year_end}.xlsx Descargado correctamente')
except:
    print(f'Error al descargar {tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{hoja_extraer}_{year_start}-{year_end}.xlsx:{e}')

In [ ]:
#==================== CAMBIAR NOMBRES BM================================
t_empresa = {0:'0', 1:'1', 2:'2', 3:'3',4:'4'}
hoja_extraer = 'gyp'
for n in range(0,5):
    t_empr = t_empresa[n]
    ruta_descarga = f"C:\\Users\\USER\\Documents\\GITHUB\\"
    data_generada = f'{ruta_descarga}{tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{hoja_extraer}_{nombre_seleccion}_{year_start}.{month1:02}-{year_end}.{month2:02}'
    try:
        print("Archivo leído correctamente después de guardar como .XLS.")
        data_total = pd.read_excel(f'{data_generada}.xlsx')
        if t_empr == '0':
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace(' DEL PERÚ', '', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace(' PERÚ', '', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO FALABELLA ', 'BANCO FALABELLA', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('FALABELLA  SA', 'BANCO FALABELLA', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO SANTANDER ', 'BANCO SANTANDER', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO SUDAMERICANO ', 'BANCO SUDAMERICANO', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO SUDAMERICANOCON SUCURSALES EN EL EXTERIOR', 'BANCO SUDAMERICANO CON SUCURSALES EN EL EXTERIOR', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO DE CHINA', 'BANK OF CHINA', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANK OF CHINA ', 'BANK OF CHINA', regex=False)    
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('INTERBANK  CON SUCURSALES EN EL EXTERIOR', 'INTERBANK CON SUCURSALES EN EL EXTERIOR', regex=False)        
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('SCOTIABANK  CON SUCURSALES EN EL EXTERIOR', 'SCOTIABANK CON SUCURSALES EN EL EXTERIOR', regex=False)        
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('SANTANDER  SA', 'BANCO SANTANDER', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('MÚLTIPE', 'MÚLTIPLE', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('INCLUYENDO', 'INCLUYE', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('DEUTSCHE BANK PER', 'DEUTSCHE BANK', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO SANTANDERCENTRAL HISPANO', 'BANCO SANTANDER CENTRAL HISPANO', regex=False)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
        elif t_empr == '1':
            #==================== CAMBIAR NOMBRES EMPRESAS FINANCIERAS================================
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('AMÉRIKA', 'AMERIKA', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('EDYFICAR', 'EDIFICAR', regex=False)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
        elif t_empr == '3':
            #==================== CAMBIAR NOMBRES CAJAS RURALES================================
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('PERU', 'PERÚ', regex=False)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
        elif t_empr == '4':
            #==================== CAMBIAR NOMBRES EMPRESAS DE CREDITOS================================
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace(r'\bTOTAL EMPRESAS DE CRÉDITO\b(?!S)', 'TOTAL EMPRESAS DE CRÉDITOS', regex=True)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('TOTAL ENTIDADES DE DESARROLLO DE LA PEQUEÑA Y MICRO EMPRESA', 'TOTAL EMPRESAS DE CRÉDITOS', regex=True)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('TOTAL SERVICIOS FINANCIEROS', 'SERVICIOS FINANCIEROS TOTAL', regex=True)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('SERVICIOS FINANCIEROS TOTAL', 'SERVICIOS FINANCIEROS', regex=True)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
        else:
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('CAJA MUNICIPAL DE CRÉDITO POPULAR LIMA', 'CMCP LIMA', regex=True)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
    except:
        print(f'Archivo {data_generada} no encontrado ')
    continue

#### 7.2 ESTADO DE SITUACION FINANCIERA (BALANCE GENERAL)

In [ ]:
#SOLO PARA ESTADOS FINANCIEROS - BG
tipo_empresa = {'0': 'Banca Múltiple', '1': 'Empresas Financieras', '2': 'Caja Municipal', '3': 'Caja Rural' , 
                '4': 'Empresas de Créditos' }
carpeta = {'0': 'BM', '1': 'EF', '2': 'CMAC', '3': 'CRAC' , '4': 'EC' }
mes_corto = {0:'en', 1:'fe', 2:'ma', 3:'ab', 4:'my', 5:'jn', 6:'jl', 7:'ag', 8:'se', 9:'oc', 10:'no', 11:'di'}
#ruta_base = f"D:\\SBS\\EMP SF\\{carpeta[t_empr]}"
pd.set_option('display.float_format', '{:.2f}'.format)
ctas_extraer = ['DISPONIBLE','Vigentes','Refinanciados y Reestructurados','Atrasados', 
                'Vigentes*','Refinanciados y Reestructurados*','Atrasados*', 
                'Vigentes**','Refinanciados y Reestructurados**','Atrasados**', 
                'Provisiones', 'TOTAL ACTIVO','T O T A L A C T I V O','TOTAL PASIVO']
hoja_extraer = 'bg'
ruta_base = f"D:\\SBS\\EMP SF\\{carpeta[t_empr]}"
data = {}
# Verificar si el archivo existe
for year in range(int(year_start),int(year_end) +1):
    for m in range(12):
        fecha = f"{year}{str(m+1).zfill(2)}01"
        fecha1 = pd.to_datetime(fecha, format='%Y%m%d')
        if (fecha1 >= pd.to_datetime(f'{year_start}-{month1}-01')) and (fecha1 <= pd.to_datetime(f'{year_end}-{month2}-01')):
            nombre_archivo = f'{tipo_empresa_descargar[selec_archivo][t_empr]}-{mes_corto[m]}{year}.XLS' 
            ruta_archivo = f'{ruta_base}\\{tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{year}\\{nombre_archivo}'
            guardar_como_xls(ruta_archivo)
            archivo = pd.read_excel(ruta_archivo, engine='xlrd', sheet_name = 0)
            try:
                print("Archivo leído correctamente después de guardar como .XLS.")
            except Exception as e:
                print(f'Error al cargar {nombre_archivo}:{e}')

            if pd.isna(archivo.iloc[1,0]) == True:
                archivo = archivo.iloc[:,1:]
            ## ==============================================TRABAJANDO CON EL DF IMPORTADO =========================================================
            #####  PARTE 01 : IDENTIFICAR LAS FILAS QUE VAMOS A TRABAJAR
            
            #Aplicamos la función limpiar texto, para eliminar los espacios innecesarios
            archivo = archivo.applymap(limpiar_texto)
            #Eliminamos todos los valores nan del df archivo
            archivo = archivo.dropna(axis=1, how='all')
            #Determinamos el indice de la palabra Activo
            indice1 = archivo[archivo.iloc[:, 0].str.contains("DISPONIBLE", na=False, case=False)].index.tolist()
            archivo_base1= archivo.copy()         
            #Colocamos toda la fila del indice del Activo como nombres de columnas
            archivo.columns =archivo.iloc[indice1[0]-3]
            #Cambiamos los nombres de las columnas a MAYUSCULAS 
            archivo.columns = [str(nombre).upper().strip().replace("  ", " ") 
                                if isinstance(nombre, str) 
                                else nombre for nombre in archivo.columns]
               
            archivo.columns = [np.nan if col == 'ACTIVO' else col for col in archivo.columns]
                
            #Nos quedamos con las filas debajo del indice Activo
            archivo = archivo.iloc[indice1[0]-1:,:]
            #Establecemos una lista de todos los nombres de las columnas
            col_names = archivo.columns.tolist()
            #Agregamos el nombre CUENTAS y quitamos el nombre Activo
            col_names1 = ['CUENTAS'] + col_names[1:]
            archivo.columns = col_names1
            archivodd = archivo.copy()
            #Solo nos quedamos con las filas de las cuentas que estan en la lista "ctas_extraer"
            archivo = archivo[archivo.CUENTAS.isin(ctas_extraer)]
            #Reseteamos el indice
            archivo = archivo.reset_index(drop=True)

            #####  PARTE 02 : IDENTIFICAR LAS COLUMNAS QUE VAMOS A TRABAJAR
            archivo3 = archivo.copy()
            #Del archivo incial, nos quedamos que las columnas sean la fila de los MN,ME,MT
            archivo2 = archivo_base1.copy()
            archivo2.columns = archivo2.iloc[indice1[0]-2]
            #Establecemos el criterio de seleccion
            nombre_seleccion = 'TOTAL'
            columnas_seleccion = []
            columnas_seleccion = [i for i, col in enumerate(archivo2.columns) if pd.notna(col) and col == nombre_seleccion]
            columnas_conservar = [0] + columnas_seleccion

            ### CASO ESPECIAL: EN CASO FALTE UN ENTIDAD QUE NO TENGA LOS VALORES: MN, ME Ó TOTAL
            #### Primero, verificamos la cantidad de entidades
            entidades = archivo3.columns.tolist()
            entidades = pd.Series(entidades).dropna(how='all').tolist()
            entidades1 = entidades[1:]
            entidades_index = [i for i, col in enumerate(archivo3.columns) if pd.notna(col)]
            #### Segundo, creamos una lista de todo el orden cada entidad en base a lo reportado anteriormente      
            col_MN = [0]
            col_ME = [0]
            col_TOTAL = [0]

            for m in range(1,len(entidades_index)):
                if (archivo_base1.iloc[indice1[0]:indice1[0]+1,entidades_index[m]].values[0] !=0) or pd.notna(archivo_base1.iloc[indice1[0]:indice1[0]+1,entidades_index[m]].values[0]) == True:
                    valor1 = entidades_index[m]
                    col_MN.append(valor1)
                    valor2 = valor1+1
                    col_ME.append(valor2)
                    valor3 = valor2+1
                    col_TOTAL.append(valor3)
                else:
                    valor1 = entidades_index[m]+1
                    col_MN.append(valor1)
                    valor2 = valor1+1
                    col_ME.append(valor2)
                    valor3 = valor2+1
                    col_TOTAL.append(valor2)
            #### Tercero, evaluamos para 
            #Establecemos las columnas seleccionadas
            moneda = {}
            moneda = {'MN':col_MN, 'ME':col_ME, 'TOTAL':col_TOTAL}
            if len(columnas_conservar) < len(moneda[nombre_seleccion]):
                archivo = archivo.iloc[:,moneda[nombre_seleccion]]
            else:
                archivo = archivo.iloc[:,columnas_conservar]

            #####  PARTE 03 : NOS QUEDAMOS CON LAS FILAS Y COLUMNAS SELECCIONADAS
            #Ahora nos quedamos solo con los nombres de las entidades financieras
            archivo.columns = entidades
            archivo['FECHA'] = fecha1
            archivo = archivo.melt(id_vars=['CUENTAS','FECHA'], var_name=tipo_empresa[t_empr], value_name='Valor')
            archivofinal = archivo.pivot_table(index=[tipo_empresa[t_empr],'FECHA'], columns='CUENTAS', values='Valor', aggfunc='sum')
            archivofinal.reset_index(inplace = True)
            archivofinal.columns = archivofinal.columns.str.replace('*', '', regex=False)
            archivofinal.iloc[:, 0] = archivofinal.iloc[:, 0].apply(limpiar_nombre)
            data[fecha] = archivofinal

data_total = pd.concat(data.values(),axis=0)
columnas_orden = [data_total.columns[1]] + [data_total.columns[0]] + list(data_total.columns[2:])
data_total = data_total[columnas_orden]
ruta_descarga = f"C:\\Users\\USER\\Documents\\GITHUB\\"
data_generada = f'{ruta_descarga}{tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{hoja_extraer}_{nombre_seleccion}_{year_start}.{month1:02}-{year_end}.{month2:02}.xlsx'
data_total.to_excel(data_generada, index=False)
try:
    print(f'Archivo {tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{hoja_extraer}_{year_start}-{year_end}.xlsx Descargado correctamente')
except:
    print(f'Error al descargar {tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{hoja_extraer}_{year_start}-{year_end}.xlsx:{e}')

In [ ]:
#==================== CAMBIAR NOMBRES BM================================
t_empresa = {0:'0', 1:'1', 2:'2', 3:'3',4:'4'}
hoja_extraer = 'bg'
for n in range(0,5):
    t_empr = t_empresa[n]
    ruta_descarga = f"C:\\Users\\USER\\Documents\\GITHUB\\"
    data_generada = f'{ruta_descarga}{tipo_empresa[t_empr]}-{archivo_descargado[selec_archivo]}_{hoja_extraer}_{nombre_seleccion}_{year_start}.{month1:02}-{year_end}.{month2:02}'
    try:
        print("Archivo leído correctamente después de guardar como .XLS.")
        data_total = pd.read_excel(f'{data_generada}.xlsx')
        if t_empr == '0':
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace(' DEL PERÚ', '', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace(' PERÚ', '', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO FALABELLA ', 'BANCO FALABELLA', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('FALABELLA  SA', 'BANCO FALABELLA', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO SANTANDER ', 'BANCO SANTANDER', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO SUDAMERICANO ', 'BANCO SUDAMERICANO', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO SUDAMERICANOCON SUCURSALES EN EL EXTERIOR', 'BANCO SUDAMERICANO CON SUCURSALES EN EL EXTERIOR', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO DE CHINA', 'BANK OF CHINA', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANK OF CHINA ', 'BANK OF CHINA', regex=False)    
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('INTERBANK  CON SUCURSALES EN EL EXTERIOR', 'INTERBANK CON SUCURSALES EN EL EXTERIOR', regex=False)        
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('SCOTIABANK  CON SUCURSALES EN EL EXTERIOR', 'SCOTIABANK CON SUCURSALES EN EL EXTERIOR', regex=False)        
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('SANTANDER  SA', 'BANCO SANTANDER', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('MÚLTIPE', 'MÚLTIPLE', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('INCLUYENDO', 'INCLUYE', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('DEUTSCHE BANK PER', 'DEUTSCHE BANK', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('BANCO SANTANDERCENTRAL HISPANO', 'BANCO SANTANDER CENTRAL HISPANO', regex=False)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
        elif t_empr == '1':
            #==================== CAMBIAR NOMBRES EMPRESAS FINANCIERAS================================
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('AMÉRIKA', 'AMERIKA', regex=False)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('EDYFICAR', 'EDIFICAR', regex=False)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
        elif t_empr == '3':
            #==================== CAMBIAR NOMBRES CAJAS RURALES================================
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('PERU', 'PERÚ', regex=False)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
        elif t_empr == '4':
            #==================== CAMBIAR NOMBRES EMPRESAS DE CREDITOS================================
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace(r'\bTOTAL EMPRESAS DE CRÉDITO\b(?!S)', 'TOTAL EMPRESAS DE CRÉDITOS', regex=True)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('TOTAL ENTIDADES DE DESARROLLO DE LA PEQUEÑA Y MICRO EMPRESA', 'TOTAL EMPRESAS DE CRÉDITOS', regex=True)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('TOTAL SERVICIOS FINANCIEROS', 'SERVICIOS FINANCIEROS TOTAL', regex=True)
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('SERVICIOS FINANCIEROS TOTAL', 'SERVICIOS FINANCIEROS', regex=True)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
        else:
            data_base = data_total.copy()
            data_base.iloc[:, 1] = data_base.iloc[:, 1].str.replace('CAJA MUNICIPAL DE CRÉDITO POPULAR LIMA', 'CMCP LIMA', regex=True)
            data_base.to_excel(f'{data_generada}-final.xlsx', index=False)
    except:
        print(f'Archivo {data_generada} no encontrado ')
    continue